<a href="https://colab.research.google.com/github/vrhughes/DS4002-Project3/blob/main/VGG16.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Basic importing
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
# Importing from example code
import os
from glob import glob
import tensorflow as tf
import keras

from tensorflow.keras.layers import Activation, BatchNormalization, Conv2D, Dense, Dropout, Flatten, MaxPool2D
from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers import Adam, RMSprop, SGD
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from tensorflow.keras.applications import VGG16

from PIL import Image

In [3]:
# Getting kaggle path
import kagglehub

# Download latest version
path = kagglehub.dataset_download("puneet6060/intel-image-classification")

print("Path to dataset files:", path)

Path to dataset files: /kaggle/input/intel-image-classification


In [4]:
# paths of data
test_path = '/kaggle/input/intel-image-classification/seg_test/seg_test'
train_path = '/kaggle/input/intel-image-classification/seg_train/seg_train'

In [12]:
# Checking what is in the folders
for folder in  os.listdir(train_path):
    files = glob(pathname= str(train_path + '/' + folder + '/*.jpg'))
    print(f'Found {len(files)} in folder {folder}')


Found 2512 in folder mountain
Found 2382 in folder street
Found 2191 in folder buildings
Found 2274 in folder sea
Found 2271 in folder forest
Found 2404 in folder glacier


In [11]:
train_path+'mountain'

'/kaggle/input/intel-image-classification/seg_train/seg_trainmountain'

In [10]:
os.listdir(train_path)

['mountain', 'street', 'buildings', 'sea', 'forest', 'glacier']